Import necessary libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Scrape web page and print df

In [13]:
# idea for web scraping: scrape audible best-selling books
# Put the link to Audible Best-Sellers page here no matter the category
# URL = "https://www.audible.com/adblbestsellers?searchCategory=18573518011&ref=a_adblbests_l1_catRefs_10&pf_rd_p=2ea8d46b-3372-49db-8ad4-77416e49695f&pf_rd_r=Z5CN827PYCK5J0XNQ8SM"
# URL = "https://www.audible.com/adblbestsellers?searchCategory=18580540011&ref=a_adblbests_l1_catRefs_20&pf_rd_p=2ea8d46b-3372-49db-8ad4-77416e49695f&pf_rd_r=1DF75X5DWF19XQYMVZ07"
# These three URLs below are working in Black Friday's sale
URL = "https://www.audible.com/search?sort=popularity-rank&ref=a_search_l1_catBackAll&pf_rd_p=daf0f1c8-2865-4989-87fb-15115ba5a6d2&pf_rd_r=S4VEHTG7BAZHCPTD9YB7"
URL = "https://www.audible.com/search?crid=31E3J6JGY980O&i=na-audible-us&k=brandon+sanderson&keywords=brandon+sanderson&ref-override=a_search_t1_header_search&sort=pubdate-desc-rank&sprefix=brand%2Cna-audible-us%2C170&url=search-alias%3Dna-audible-us&ref=a_search_c1_sort_1&pf_rd_p=073d8370-97e5-4b7b-be04-aa06cf22d7dd&pf_rd_r=VRAHHY22TC5WB0JFHJFR"
# URL = "https://www.audible.com/author/Brandon-Sanderson/B001IGFHW6?ref=a_search_c3_lAuthor_1_7_1&pf_rd_p=83218cca-c308-412f-bfcf-90198b687a2f&pf_rd_r=B202FRY6C8JQHCG7KD4Q"
from scraping import audible_scraper

df = audible_scraper(URL)
df.head()

,title,description,author,rating,rating_count,reg_price,audio_len,language
0,The Shepherd's Crown,"Discworld, Book 41",Terry Pratchett,None,Not rated yet,$34.90,Not Yet Known,English
1,I Shall Wear Midnight,"Discworld, Book 38",Terry Pratchett,None,Not rated yet,$34.90,Not Yet Known,English
2,Curso de escritura creativa [Creative Writing ...,None,Brandon Sanderson,None,Not rated yet,$21.73,10 hrs and 48 mins,Spanish
3,Spare,None,Prince Harry The Duke of Sussex,None,Not rated yet,$50.40,Not Yet Known,English
4,Dark One: Forgotten,None,Brandon Sanderson,None,Not rated yet,$20.99,5 hrs,English


In [4]:
df

,title,description,author,rating,rating_count,reg_price,audio_len,language
0,The Lost Metal,A Mistborn Novel,Brandon Sanderson,5 out of 5 stars,"2,530 ratings",$31.18,18 hrs and 46 mins,English
1,The Final Empire,Mistborn Book 1,Brandon Sanderson,5 out of 5 stars,"80,745 ratings",$33.94,24 hrs and 39 mins,English
2,The Way of Kings,"The Stormlight Archive, Book 1",Brandon Sanderson,5 out of 5 stars,"94,813 ratings",$26.99,45 hrs and 30 mins,English
3,The Alloy of Law,A Mistborn Novel,Brandon Sanderson,4.5 out of 5 stars,"35,388 ratings",$12.94,10 hrs and 48 mins,English
4,The Well of Ascension,"Mistborn, Book 2",Brandon Sanderson,5 out of 5 stars,"61,625 ratings",$34.90,28 hrs and 56 mins,English
5,Words of Radiance,"The Stormlight Archive, Book 2",Brandon Sanderson,5 out of 5 stars,"76,441 ratings",$38.90,48 hrs and 13 mins,English
6,Oathbringer,None,Brandon Sanderson,5 out of 5 stars,"57,425 ratings",$42.90,55 hrs and 5 mins,English
7,Rhythm of War,Book Four of The Stormlight Archive,Brandon Sanderson,5 out of 5 stars,"32,247 ratings",$29.99,57 hrs and 26 mins,English
8,The Hero of Ages,"Mistborn, Book 3",Brandon Sanderson,5 out of 5 stars,"62,208 ratings",$34.90,27 hrs and 25 mins,English
9,Shadows of Self,None,Brandon Sanderson,4.5 out of 5 stars,"27,295 ratings",$17.92,12 hrs and 37 mins,English


In [5]:
from uncategorized import check_book_availability

check_book_availability("The Lost Metal", df)
check_book_availability("fjdslkf", df)
check_book_availability("Centers of Gravity", df)
check_book_availability("The Shepherd's Crown", df)


The book with the title 'The Lost Metal' is present in the catalogue
and is available for buying

The book with the title 'fjdslkf' isn't present in the catalogue

The book with the title 'Centers of Gravity' isn't present in the catalogue

The book with the title 'The Shepherd's Crown' isn't present in the catalogue


Create connection to MySQL

In [6]:
# later put related to MySQL code and imports into a separate file
from sqlalchemy import create_engine
from env_vars import DB_USER, DB_PASSWORD

def establish_connection():
    DB_TO_WORK_WITH = "audible_books_db"
    DB_HOST = "localhost:3306"
    return create_engine(
        f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_TO_WORK_WITH}",
        pool_recycle=3600)

engine = establish_connection()

In [7]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String

def create_books_table():
    metadata_obj = MetaData()
    books = Table(
        "books",
        metadata_obj,
        Column("id", Integer, primary_key=True, autoincrement=True),
        Column("title", String(255), nullable=False),
        Column("description", String(255)),
        Column("author", String(255), nullable=False),
        Column("rating", String(31)),
        Column("rating_count", String(15)),
        Column("reg_price", String(15)),
        Column("audio_len", String(31)),
        Column("language", String(31))
    )
    metadata_obj.create_all(engine)


Load df to db

In [22]:
from sqlalchemy.exc import ProgrammingError

# If you load to db straightaway, you can get dublicate values
# To solve this problem, read everything from your DB table into df,
# combine with fresh data, remove dublicates,
# truncate your DB table, and load info back to DB
# Warning: may be slow with big amount of data
def load_to_DB(input_df):
    try:
        all_info_from_db = pd.read_sql("select * from books;", con=engine, index_col="id")
        # combine two dfs
        df_combined = pd.concat([all_info_from_db, input_df])
        # remove dublicates and reset index
        df_combined.drop_duplicates(subset=["title", "author", "reg_price", "audio_len", "language"],
                                    ignore_index=True, inplace=True)
        # Truncate table 'books'
        engine.connect().execute("TRUNCATE TABLE books")
        # load all info to DB
        df_combined.to_sql(name="books", if_exists='append', con=engine, index=False)
    except ProgrammingError:
        print("The table doesn't exist. Creating one and adding info to it right now...")
        create_books_table()
        input_df.to_sql(name="books", if_exists='append', con=engine, index=False)

load_to_DB(df)

Read from your db

In [23]:
pd.read_sql("select * from books;", con=engine, index_col="id")

,title,description,author,rating,rating_count,reg_price,audio_len,language
id,,,,,,,,
1,The Lost Metal,A Mistborn Novel,Brandon Sanderson,5 out of 5 stars,"2,530 ratings",$31.18,18 hrs and 46 mins,English
2,The Final Empire,Mistborn Book 1,Brandon Sanderson,5 out of 5 stars,"80,745 ratings",$33.94,24 hrs and 39 mins,English
3,The Way of Kings,"The Stormlight Archive, Book 1",Brandon Sanderson,5 out of 5 stars,"94,813 ratings",$26.99,45 hrs and 30 mins,English
4,The Alloy of Law,A Mistborn Novel,Brandon Sanderson,4.5 out of 5 stars,"35,388 ratings",$12.94,10 hrs and 48 mins,English
5,The Well of Ascension,"Mistborn, Book 2",Brandon Sanderson,5 out of 5 stars,"61,625 ratings",$34.90,28 hrs and 56 mins,English
6,Words of Radiance,"The Stormlight Archive, Book 2",Brandon Sanderson,5 out of 5 stars,"76,441 ratings",$38.90,48 hrs and 13 mins,English
7,Oathbringer,None,Brandon Sanderson,5 out of 5 stars,"57,425 ratings",$42.90,55 hrs and 5 mins,English
8,Rhythm of War,Book Four of The Stormlight Archive,Brandon Sanderson,5 out of 5 stars,"32,247 ratings",$29.99,57 hrs and 26 mins,English
9,The Hero of Ages,"Mistborn, Book 3",Brandon Sanderson,5 out of 5 stars,"62,208 ratings",$34.90,27 hrs and 25 mins,English
